In [1]:
# import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from datetime import datetime

<h4>02 Data Collection</h4>

In [2]:
df = []
src = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\train\train"
for file in os.listdir(src):
    df.append(pd.read_csv(os.path.join(src,file)))
# for i in df:
#     print(list(i))
df = pd.concat(df)
df = df.fillna(0)
df[-10:]

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
104,12_105,04:40:16.090733+00,44.1,31.2,0.0,47.64,0.0,0.0,0.5,3.0,38.8,22.8,0.11,5.0,21.0,95.0,87.0,105.0,0.0
105,12_106,04:42:16.132055+00,44.2,31.2,0.0,49.50,0.0,0.0,0.1,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,111.0,0.0
106,12_107,04:44:16.107327+00,45.2,30.8,0.0,50.03,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,113.0,0.0
107,12_108,04:46:16.11276+00,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
108,12_109,04:48:16.120498+00,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
109,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
110,12_111,04:52:18.185763+00,42.5,32.5,0.0,53.76,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,0.0,0.0
111,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0
112,12_113,04:56:20.078654+00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,2.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,104.0,0.0
113,12_114,04:58:16.116687+00,44.8,32.5,0.0,58.81,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,113.0,0.0


In [3]:
print(df.shape)
df.columns

(1340, 19)


Index(['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh',
       '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [4]:
# Define the function to categorize time of day
def get_time_of_day(hour):
    if 0 <= hour < 5:           # 'Early Morning'
        return 0.0
    elif 5 <= hour < 12:        # 'Morning'
        return 1.0
    elif 12 <= hour < 17:       # 'Afternoon'
        return 2.0
    elif 17 <= hour < 21:       # 'Evening'
        return 3.0
    else:                       # 'Night'
        return 4.0
    
# Convert 'time' column to datetime objects, ignoring the timezone for simplicity
df['time_parsed'] = pd.to_datetime(df['time'].str.split('+').str[0], format='%H:%M:%S.%f')
# Extract hour from the 'time_parsed' column
df['hour'] = df['time_parsed'].dt.hour
# Apply the get_time_of_day function to create 'time_of_day' column
df['time_of_day'] = df['hour'].apply(get_time_of_day)
df['time_of_day'] = df['time_of_day'].astype('float64')
df['hour'] = df['hour'].astype('float64')
# Now df includes the new 'time_of_day' column
# print(df[['time', 'time_of_day']])

In [5]:
df.describe(include='all')

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,...,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,time_parsed,hour,time_of_day
count,1340,1340,1340.000000,1340.000000,1340.000000,1340.000000,1340.0,1340.0,1340.000000,1340.000000,...,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340.000000,1340,1340.000000,1340.000000
unique,1340,1340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,01_1,01:00:50.174233+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,48.565149,28.975373,21.761828,27.479515,0.0,0.0,0.751045,2.914179,...,0.118567,6.512910,20.721642,89.968657,83.330597,40.767164,0.403731,1900-01-01 04:27:40.767920640,3.958955,0.329104
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1900-01-01 00:26:03.264438,0.000000,0.000000
25%,NaN,NaN,41.800000,25.900000,0.000000,5.475000,0.0,0.0,0.100000,1.000000,...,0.110000,5.000000,21.000000,94.000000,87.000000,23.000000,0.000000,1900-01-01 02:30:48.188879104,2.000000,0.000000
50%,NaN,NaN,45.700000,29.600000,0.000000,11.005000,0.0,0.0,0.600000,2.000000,...,0.110000,7.000000,22.000000,97.000000,89.000000,29.000000,0.000000,1900-01-01 04:00:46.233681408,4.000000,0.000000
75%,NaN,NaN,52.725000,32.300000,65.530000,55.992500,0.0,0.0,1.100000,4.000000,...,0.120000,7.400000,23.000000,100.000000,93.000000,52.000000,1.000000,1900-01-01 05:37:00.989231360,5.000000,1.000000
max,NaN,NaN,97.300000,38.400000,65.530000,65.520000,0.0,0.0,5.000000,7.000000,...,0.210000,8.300000,33.000000,123.000000,116.000000,123.000000,1.000000,1900-01-01 13:03:34.487929,13.000000,2.000000


In [6]:
df = df.drop(['index','time','11Rain Accumulate (mm)','11Rain count','time_parsed',],axis=1)

<hr>

<h4>03 Exploratory Data Analysis EDA</h4>

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1340 entries, 0 to 113
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   1Air Humidity           1340 non-null   float64
 1   1AirTemperature         1340 non-null   float64
 2   1LightHigh              1340 non-null   float64
 3   1LightLow               1340 non-null   float64
 4   12WindSpeed             1340 non-null   float64
 5   13WindDirection         1340 non-null   float64
 6   23SoilWC ค.ชื้นดิน      1340 non-null   float64
 7   23SoilTemp.อุณหภูมิดิน  1340 non-null   float64
 8   23SoilEC                1340 non-null   float64
 9   23SoilpH                1340 non-null   float64
 10  26N                     1340 non-null   float64
 11  26P                     1340 non-null   float64
 12  26K                     1340 non-null   float64
 13  29SoilTension           1340 non-null   float64
 14  flow1                   1340 non-null   float6

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
1Air Humidity,1340.0,48.565149,11.594615,0.0,41.800,45.700,52.7250,97.30
1AirTemperature,1340.0,28.975373,4.606203,0.0,25.900,29.600,32.3000,38.40
1LightHigh,1340.0,21.761828,30.873723,0.0,0.000,0.000,65.5300,65.53
1LightLow,1340.0,27.479515,24.924393,0.0,5.475,11.005,55.9925,65.52
12WindSpeed,1340.0,0.751045,0.746727,0.0,0.100,0.600,1.1000,5.00
13WindDirection,1340.0,2.914179,2.202777,0.0,1.000,2.000,4.0000,7.00
23SoilWC ค.ชื้นดิน,1340.0,34.977164,19.713135,0.0,31.400,36.900,42.5000,80.20
23SoilTemp.อุณหภูมิดิน,1340.0,21.970224,1.738520,0.0,21.500,22.000,23.1000,23.70
23SoilEC,1340.0,0.118567,0.019267,0.0,0.110,0.110,0.1200,0.21
23SoilpH,1340.0,6.512910,1.240398,0.0,5.000,7.000,7.4000,8.30


In [10]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

In [11]:
def remove_outliers(df):
    cleaned_data = df.copy()
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            cleaned_data = cleaned_data[(cleaned_data[col] >= lower_bound) & (cleaned_data[col] <= upper_bound)]
    return cleaned_data

# Remove outliers from the dataset
df = remove_outliers(df)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
1Air Humidity,556.0,45.477338,7.901563,32.30,38.9000,45.30,50.500,69.10
1AirTemperature,556.0,28.678597,4.708653,17.70,25.1000,28.90,32.025,38.40
1LightHigh,556.0,33.236439,32.791110,0.00,0.0000,65.53,65.530,65.53
1LightLow,556.0,21.230773,23.616174,0.03,4.4875,8.50,46.370,65.52
12WindSpeed,556.0,0.769424,0.645232,0.00,0.3000,0.60,1.200,2.60
13WindDirection,556.0,3.008993,2.285463,0.00,1.0000,2.00,5.000,7.00
23SoilWC ค.ชื้นดิน,556.0,37.655755,5.749712,29.80,33.2000,36.90,39.700,58.70
23SoilTemp.อุณหภูมิดิน,556.0,21.395504,1.076111,19.90,20.4000,21.50,21.900,23.50
23SoilEC,556.0,0.112212,0.004448,0.11,0.1100,0.11,0.110,0.13
23SoilpH,556.0,7.002518,0.738804,4.80,6.8000,7.10,7.400,8.20


In [12]:
# # Pair Plot
# sns.pairplot(df)
# plt.show()

<hr>

<h4>04 Feature Engineering</h4>

In [13]:
print(df.columns)
df

Index(['1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow',
       '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน',
       '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K',
       '29SoilTension', 'flow1', 'hour', 'time_of_day'],
      dtype='object')


,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,hour,time_of_day
49,48.0,32.3,0.0,62.16,0.0,2.0,58.5,23.4,0.12,7.8,25.0,104.0,97.0,22.0,0.0,2.0,0.0
50,45.7,33.0,0.0,61.97,0.0,4.0,57.9,23.3,0.12,7.8,25.0,103.0,96.0,21.0,0.0,2.0,0.0
51,44.8,34.4,0.0,62.74,0.0,2.0,53.3,23.4,0.12,7.8,25.0,103.0,96.0,21.0,0.0,2.0,0.0
52,45.5,34.4,0.0,63.44,0.0,3.0,52.1,23.4,0.12,7.8,24.0,103.0,96.0,21.0,0.0,2.0,0.0
53,47.3,34.2,0.0,63.31,0.0,2.0,52.3,23.4,0.12,7.9,24.0,103.0,96.0,21.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,47.4,30.0,0.0,53.43,0.9,7.0,49.3,23.0,0.12,4.9,22.0,97.0,90.0,0.0,1.0,2.0,0.0
62,46.9,29.5,0.0,44.12,0.6,2.0,42.2,22.9,0.12,4.9,21.0,96.0,89.0,0.0,0.0,3.0,0.0
86,46.3,30.4,0.0,48.35,1.7,2.0,37.4,22.9,0.12,5.0,21.0,95.0,88.0,0.0,0.0,4.0,0.0
103,45.0,31.0,0.0,45.51,0.6,6.0,36.9,22.9,0.11,5.0,21.0,95.0,87.0,0.0,0.0,4.0,0.0


In [14]:
dfx = df
dfx.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1,hour,time_of_day
1Air Humidity,1.0,-0.6905,-0.4881,0.009322,0.1483,-0.1137,-0.3174,0.04584,-0.006159,-0.06994,-0.09476,-0.119,-0.1032,0.1051,0.5593,-0.2991,-0.09044
1AirTemperature,-0.6905,1.0,0.4099,-0.02208,-0.3701,0.1326,0.39,0.5368,0.07944,0.1157,0.3221,0.3371,0.3273,-0.1353,-0.4684,0.378,0.06415
1LightHigh,-0.4881,0.4099,1.0,-0.7099,0.05202,0.0007311,0.1314,0.07518,-0.2055,0.3786,-0.03526,-0.04843,-0.04214,-0.2946,-0.5109,0.2811,-0.02105
1LightLow,0.009322,-0.02208,-0.7099,1.0,-0.01126,-0.07412,0.1876,0.03009,0.3505,-0.4552,0.3834,0.4081,0.3963,0.3719,0.07409,-0.1335,-0.01558
12WindSpeed,0.1483,-0.3701,0.05202,-0.01126,1.0,-0.03745,-0.08021,-0.159,0.04747,-0.1172,-0.005389,0.001055,0.002337,0.1249,0.07131,0.001699,0.03655
13WindDirection,-0.1137,0.1326,0.0007311,-0.07412,-0.03745,1.0,0.1117,0.03108,-0.01082,0.06572,-0.1109,-0.1015,-0.08293,-0.1184,0.02314,-0.1568,-0.1535
23SoilWC ค.ชื้นดิน,-0.3174,0.39,0.1314,0.1876,-0.08021,0.1117,1.0,0.257,0.5281,0.1369,0.3414,0.4142,0.4016,-0.2049,-0.2515,-0.0314,-0.02952
23SoilTemp.อุณหภูมิดิน,0.04584,0.5368,0.07518,0.03009,-0.159,0.03108,0.257,1.0,0.2709,0.1404,0.6282,0.6215,0.6266,0.02241,-0.008177,0.241,0.1119
23SoilEC,-0.006159,0.07944,-0.2055,0.3505,0.04747,-0.01082,0.5281,0.2709,1.0,-0.4585,0.5823,0.6381,0.6446,0.4709,0.2013,0.01455,0.0641
23SoilpH,-0.06994,0.1157,0.3786,-0.4552,-0.1172,0.06572,0.1369,0.1404,-0.4585,1.0,-0.2276,-0.2384,-0.2368,-0.6438,-0.2336,0.01138,-0.01824


In [15]:
feature_sort = dict(dfx.corr()['23SoilWC ค.ชื้นดิน'])
sorted_x = dict(sorted(feature_sort.items(), key=lambda item: item[-1],reverse=True))
sorted_x

{'23SoilWC ค.ชื้นดิน': 1.0,
 '23SoilEC': 0.5281081566437361,
 '26P': 0.41420988666284797,
 '26K': 0.4016029577965687,
 '1AirTemperature': 0.39004364171796796,
 '26N': 0.3414103062207534,
 '23SoilTemp.อุณหภูมิดิน': 0.2570289511234114,
 '1LightLow': 0.18762615074765268,
 '23SoilpH': 0.13691602826742844,
 '1LightHigh': 0.13137198941400632,
 '13WindDirection': 0.11166979877548296,
 'time_of_day': -0.02951674321066998,
 'hour': -0.031398999810761816,
 '12WindSpeed': -0.08021495660255422,
 '29SoilTension': -0.20492669732133947,
 'flow1': -0.25150346711170646,
 '1Air Humidity': -0.31744692959066034}

In [16]:
print(list(sorted_x.keys())[1:])

['23SoilEC', '26P', '26K', '1AirTemperature', '26N', '23SoilTemp.อุณหภูมิดิน', '1LightLow', '23SoilpH', '1LightHigh', '13WindDirection', 'time_of_day', 'hour', '12WindSpeed', '29SoilTension', 'flow1', '1Air Humidity']


<h5>Feature  selectin</h5>

In [17]:
# X = dfx[['1Air Humidity', '1AirTemperature', '1LightHigh',
#                 '1LightLow','12WindSpeed',
#                 '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
#                 '29SoilTension', 'flow1']]

# X = dfx[['26P', '26K', '26N', '1AirTemperature', '23SoilTemp.อุณหภูมิดิน','23SoilpH', '1LightLow', 'time_of_day']]   'XGBoost': 0.8941797429865055,   
# X = dfx[['23SoilEC', '26P', '26K', '1AirTemperature', '23SoilpH', '1LightLow']]       #  'XGBoost': 1.1382227172851562,    
# X = dfx[['23SoilEC', '26P', '26K', '1AirTemperature', '23SoilpH', '1LightLow','time_of_day']]       # 'Gradient Boosting Regression2 (lgb_best)': 0.9417558774252336
# X = dfx[['23SoilEC','26P','26K','26N','1AirTemperature','23SoilTemp.อุณหภูมิดิน','23SoilpH','1LightLow']]  #'XGBoost': 1.1545134582519532,
# X = dfx[['23SoilEC','26P','26K','26N','1AirTemperature','23SoilTemp.อุณหภูมิดิน','23SoilpH','1LightLow','time_of_day']]  # 'XGBoost (xgb_best)': 0.8907829544760962,
# X = dfx[['23SoilEC','26P','26K','26N','1AirTemperature','23SoilTemp.อุณหภูมิดิน','23SoilpH','1LightLow','hour']]    # 'Gradient Boosting Regression2 (lgb_best)': 0.8217750462012927,
X = dfx[['23SoilEC','26P', '26K', '26N', '1AirTemperature', '23SoilTemp.อุณหภูมิดิน','23SoilpH', '1LightLow', 'hour']]   
y = df[['23SoilWC ค.ชื้นดิน']]

In [18]:
X

,23SoilEC,26P,26K,26N,1AirTemperature,23SoilTemp.อุณหภูมิดิน,23SoilpH,1LightLow,hour
49,0.12,104.0,97.0,25.0,32.3,23.4,7.8,62.16,2.0
50,0.12,103.0,96.0,25.0,33.0,23.3,7.8,61.97,2.0
51,0.12,103.0,96.0,25.0,34.4,23.4,7.8,62.74,2.0
52,0.12,103.0,96.0,24.0,34.4,23.4,7.8,63.44,2.0
53,0.12,103.0,96.0,24.0,34.2,23.4,7.9,63.31,2.0
...,...,...,...,...,...,...,...,...,...
45,0.12,97.0,90.0,22.0,30.0,23.0,4.9,53.43,2.0
62,0.12,96.0,89.0,21.0,29.5,22.9,4.9,44.12,3.0
86,0.12,95.0,88.0,21.0,30.4,22.9,5.0,48.35,4.0
103,0.11,95.0,87.0,21.0,31.0,22.9,5.0,45.51,4.0


In [19]:
y

,23SoilWC ค.ชื้นดิน
49,58.5
50,57.9
51,53.3
52,52.1
53,52.3
...,...
45,49.3
62,42.2
86,37.4
103,36.9


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.179, random_state = 99)
X_train.shape, X_test.shape

((456, 9), (100, 9))

<h5>Scaleing?</h5>

In [22]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [23]:
# X_train = X_train_scaled
# X_test  = X_test_scaled

In [24]:
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# X_train = X_train_scaled
# X_test  = X_test_scaled

# knn_param_grid = {
#     'n_neighbors': [3, 5, 7],
#     'weights': ['uniform', 'distance'],
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
# }
# knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
# knn_grid_search.fit(X_train, y_train)
# knn_best = knn_grid_search.best_estimator_
# knn_predictions = knn_best.predict(X_test)
# knn_mae = mean_absolute_error(y_test, knn_predictions)
# print(f'KNN Regression MAE: {knn_mae}, Best Params: {knn_grid_search.best_params_}')

In [25]:
# dt_best.predict(X_test)

<hr>

<h4>05 Model Selection</h4>

In [26]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from pygam import LinearGAM, s, f
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler


<hr>
<hr>

In [38]:
knn_param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}
knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
scaler = StandardScaler()
knn_grid_search.fit(scaler.fit_transform(X_train), y_train)
knn_best = knn_grid_search.best_estimator_
# print("Use: X_test_scaled = scaler.transform(X_test) ")
knn_predictions = knn_best.predict(scaler.transform(X_test))
knn_mae = mean_absolute_error(y_test, knn_predictions)

<h4>06 Model Training</h4>

In [27]:
x = []
src2 = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test"
for file in os.listdir(src2):
    print(os.path.join(src2,file))
    x.append(pd.read_csv(os.path.join(src2,file)))
x = pd.concat(x)
x = x.fillna(0)

# Define the function to categorize time of day
def get_time_of_day(hour):
    if 0 <= hour < 5:           # 'Early Morning'
        return 0.0
    elif 5 <= hour < 12:        # 'Morning'
        return 1.0
    elif 12 <= hour < 17:       # 'Afternoon'
        return 2.0
    elif 17 <= hour < 21:       # 'Evening'
        return 3.0
    else:                       # 'Night'
        return 4.0

# Convert 'time' column to datetime objects, ignoring the timezone for simplicity
x['time_parsed'] = pd.to_datetime(x['time'].str.split('+').str[0], format='%H:%M:%S.%f')

# Extract hour from the 'time_parsed' column
x['hour'] = x['time_parsed'].dt.hour

# Apply the get_time_of_day function to create 'time_of_day' column
x['time_of_day'] = x['hour'].apply(get_time_of_day)

x['time_of_day'] = x['time_of_day'].astype('float64')
x['hour'] = x['hour'].astype('float64')
# Now x includes the new 'time_of_day' column
# print(x[['time', 'time_of_day']])
x = x.drop(['index','time','11Rain Accumulate (mm)','11Rain count','time_parsed',],axis=1)

C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set02.csv
C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set09.csv


In [28]:
x = x[X.columns]
x

,23SoilEC,26P,26K,26N,1AirTemperature,23SoilTemp.อุณหภูมิดิน,23SoilpH,1LightLow,hour
0,0.11,95.0,88.0,21.0,30.7,22.4,7.9,8.79,4.0
1,0.11,95.0,88.0,21.0,31.4,22.4,7.9,8.26,4.0
2,0.11,95.0,88.0,21.0,32.2,22.3,7.9,8.66,4.0
3,0.11,95.0,88.0,21.0,33.0,22.4,7.9,8.68,4.0
4,0.11,95.0,88.0,21.0,32.9,22.4,7.9,8.86,4.0
...,...,...,...,...,...,...,...,...,...
111,0.12,110.0,103.0,27.0,35.9,23.7,4.9,53.35,7.0
112,0.12,110.0,103.0,27.0,35.9,23.7,4.9,54.13,7.0
113,0.12,110.0,103.0,27.0,35.8,23.7,4.9,54.11,7.0
114,0.12,110.0,103.0,27.0,35.6,23.7,4.9,54.26,7.0


In [49]:
def model1():
    dt_param_grid = {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    }
    dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_param_grid, cv=5, scoring='neg_mean_absolute_error')
    dt_grid_search.fit(X_train, y_train)
    dt_best = dt_grid_search.best_estimator_
    dt_predictions = dt_best.predict(X_test)
    dt_mae = mean_absolute_error(y_test, dt_predictions)
    # print(f'Decision Tree MSE: {dt_mae}, Best Params: {dt_grid_search.best_params_}')
    return dt_mae,dt_best
def model2():
    rfr = RandomForestRegressor()
    rfr_param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10]
    }
    rfr_grid_search = GridSearchCV(rfr, rfr_param_grid, cv=5, scoring='neg_mean_absolute_error')
    rfr_grid_search.fit(X_train, y_train)
    rfr_best = rfr_grid_search.best_estimator_
    rfr_predictions = rfr_best.predict(X_test)
    rfr_mae = mean_absolute_error(y_test, rfr_predictions)
    # print("Random Forest Regressor mae:", rfr_mae)
    return rfr_mae,rfr_best
def model3():
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 6, 9]
    }
    xgb_grid_search = GridSearchCV(xgb.XGBRegressor(), xgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
    xgb_grid_search.fit(X_train, y_train)
    xgb_best = xgb_grid_search.best_estimator_
    xgb_predictions = xgb_best.predict(X_test)
    xgb_mae = mean_absolute_error(y_test, xgb_predictions)
    # print(f'XGBoost MSE: {xgb_mae}, Best Params: {xgb_grid_search.best_params_}')
    return xgb_mae,xgb_best

def model4():
    gbrt = GradientBoostingRegressor()
    gbrt_param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.05, 0.1, 0.2]
        }
    gbrt_grid_search = GridSearchCV(gbrt, gbrt_param_grid, cv=5, scoring='neg_mean_absolute_error')
    gbrt_grid_search.fit(X_train, y_train)
    gbrt_best = gbrt_grid_search.best_estimator_
    gbrt_predictions = gbrt_best.predict(X_test)
    gbrt_mae = mean_absolute_error(y_test, gbrt_predictions)
    return gbrt_mae,gbrt_best
def model5():
    lgb_param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth':[3, 6, 9, -1],
        'num_leaves': [31, 62, 127, 255]
    }
    lgb_grid_search = GridSearchCV(lgb.LGBMRegressor(), lgb_param_grid, cv=5, scoring='neg_mean_absolute_error')
    lgb_grid_search.fit(X_train, y_train)
    lgb_best = lgb_grid_search.best_estimator_
    lgb_predictions = lgb_best.predict(X_test)
    lgb_mae = mean_absolute_error(y_test, lgb_predictions)
    return lgb_mae,lgb_best
def model6():
    knn_param_grid = {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
    knn_grid_search = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=5, scoring='neg_mean_absolute_error')
    scaler = StandardScaler()
    knn_grid_search.fit(scaler.fit_transform(X_train), y_train)
    knn_best = knn_grid_search.best_estimator_
    # print("Use: X_test_scaled = scaler.transform(X_test) ")
    knn_predictions = knn_best.predict(scaler.transform(X_test))
    knn_mae = mean_absolute_error(y_test, knn_predictions)
    return knn_mae,knn_best
def model7():
    param_grid = {
            "colsample_bytree": [0.3, 0.4, 0.5],
            "gamma": [0],
            "learning_rate": [0.4, 0.5, 0.6],
            "max_depth": [1, 2, 3],
            "reg_lambda": [11, 12, 13],
            "scale_pos_weight": [0],
            "subsample": [0.4, 0.5, 0.6],
    }
    grid_cv = xgb.XGBRegressor()
    grid_cv = GridSearchCV(
        grid_cv,
        param_grid,
        n_jobs=-1,
        cv=3,
        scoring='neg_mean_absolute_error'
    )
    grid_cv.fit(X_train, y_train)
    grid_cv.best_params_
    y_hat = grid_cv.predict(X_test)
    xbbl_mae = mean_absolute_error(y_test, y_hat)
    return xbbl_mae,grid_cv
def model8():
    rf = RandomForestRegressor()
    rf.fit(X_train,y_train)
    rf_baseline = rf.predict(X_test)
    rfbl_mae = mean_absolute_error(y_test, rf_baseline)
    return rfbl_mae,rf
# def model9():
#     nn_param_grid = {
#         'hidden_layer_sizes': [(50,), (100,), (50,50)],
#         'activation': ['relu', 'tanh', 'logistic'],
#         'solver': ['adam', 'lbfgs']
#     }
#     scaler = StandardScaler().fit(X_train)
#     X_train_scaled = scaler.transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
#     nn_grid_search = GridSearchCV(MLPRegressor(max_iter=1000), nn_param_grid, cv=5, scoring='neg_mean_absolute_error')
#     nn_grid_search.fit(X_train_scaled, y_train)
#     nn_best = nn_grid_search.best_estimator_
#     nn_predictions = nn_best.predict(X_test_scaled)
#     nn_mae = mean_absolute_error(y_test, nn_predictions)
#     return nn_mae,nn_best
res = {
    'dt_mae':[],
    'rfr_mae':[],
    'xgb_mae':[],
    'gbrt_mae':[],
    'lgb_mae':[],
    'knn_mae':[],
    'xbbl_mae':[],
    'rfbl_mae':[],
    # 'nn_mae':[]
}

Ex_mae = 2
for _ in range(5):
    # dt_mae,bmodel1  = model1()
    # if dt_mae<Ex_mae:
    #     Ex_mae = dt_mae
    #     best_model = bmodel1
    # res['dt_mae'].append(dt_mae)

    # rfr_mae,bmodel2 = model2()
    # if rfr_mae<Ex_mae:
    #     Ex_mae = rfr_mae
    #     best_model = bmodel2
    # res['rfr_mae'].append(rfr_mae)

    # xgb_mae,bmodel3 = model3()
    # if xgb_mae<Ex_mae:
    #     Ex_mae = xgb_mae
    #     best_model = bmodel3
    # res['xgb_mae'].append(xgb_mae)

    # gbrt_mae,bmodel4 = model4()
    # if gbrt_mae<Ex_mae:
    #     Ex_mae = gbrt_mae
    #     best_model = bmodel4
    # res['gbrt_mae'].append(gbrt_mae)

    # lgb_mae,bmodel5 = model5()
    # if lgb_mae<Ex_mae:
    #     Ex_mae = lgb_mae
    #     best_model = bmodel5
    # res['lgb_mae'].append(lgb_mae)    
    
    # knn_mae,bmodel6 = model6()
    # if knn_mae<Ex_mae:
    #     Ex_mae = knn_mae
    #     best_model = bmodel6
    # res['knn_mae'].append(knn_mae)    
    
    # xbbl_mae,bmodel7 = model7()
    # if xbbl_mae<Ex_mae:
    #     Ex_mae = xbbl_mae
    #     best_model = bmodel7
    # res['xbbl_mae'].append(xbbl_mae)    
    
    rfbl_mae,bmodel8 = model8()
    if rfbl_mae<Ex_mae:
        Ex_mae = rfbl_mae
        best_model = bmodel8
    res['rfbl_mae'].append(rfbl_mae)

In [50]:
# If knn_mae use X_test_scaled = scaler.transform(X_test)
res

{'dt_mae': [],
 'rfr_mae': [],
 'xgb_mae': [],
 'gbrt_mae': [],
 'lgb_mae': [],
 'knn_mae': [],
 'xbbl_mae': [],
 'rfbl_mae': [1.018550000000001,
  1.029370000000001,
  1.00923,
  1.0552400000000026,
  1.0246600000000026]}

In [36]:
Ex_mae

1.0194400000000023

In [37]:
best_model

RandomForestRegressor()

In [33]:
x

,23SoilEC,26P,26K,26N,1AirTemperature,23SoilTemp.อุณหภูมิดิน,23SoilpH,1LightLow,hour
0,0.11,95.0,88.0,21.0,30.7,22.4,7.9,8.79,4.0
1,0.11,95.0,88.0,21.0,31.4,22.4,7.9,8.26,4.0
2,0.11,95.0,88.0,21.0,32.2,22.3,7.9,8.66,4.0
3,0.11,95.0,88.0,21.0,33.0,22.4,7.9,8.68,4.0
4,0.11,95.0,88.0,21.0,32.9,22.4,7.9,8.86,4.0
...,...,...,...,...,...,...,...,...,...
111,0.12,110.0,103.0,27.0,35.9,23.7,4.9,53.35,7.0
112,0.12,110.0,103.0,27.0,35.9,23.7,4.9,54.13,7.0
113,0.12,110.0,103.0,27.0,35.8,23.7,4.9,54.11,7.0
114,0.12,110.0,103.0,27.0,35.6,23.7,4.9,54.26,7.0


In [46]:
# X_scaled = scaler.transform(x)  # Scale your x data

# # Assuming knn_best is your trained and optimized KNeighborsRegressor model
# y = knn_best.predict(X_scaled)  # Predict
# y = y.flatten()
# y

y =  best_model.predict(x)
y

array([37.36 , 37.455, 37.194, 37.261, 37.226, 37.196, 37.262, 37.291,
       37.188, 37.353, 37.167, 37.274, 37.181, 37.127, 37.28 , 37.148,
       37.167, 44.996, 45.543, 45.876, 46.726, 45.867, 46.132, 45.936,
       49.406, 46.273, 46.51 , 47.422, 47.494, 40.719, 46.561, 40.264,
       51.513, 52.529, 52.907, 55.821, 52.754, 52.934, 50.075, 50.638,
       41.903, 49.377, 49.377, 50.075, 52.509, 49.42 , 49.529, 49.797,
       41.789, 49.688, 49.415, 52.579, 50.81 , 45.994, 51.472, 46.525,
       46.483, 46.191, 46.428, 51.122, 42.873, 49.29 , 32.359, 37.663,
       37.668, 32.35 , 37.939, 38.02 , 37.719, 37.984, 37.733, 37.911,
       37.989, 38.154, 38.093, 37.988, 37.777, 32.655, 38.003, 37.846,
       37.681, 37.494, 37.645, 37.524, 37.777, 37.682, 37.734, 37.799,
       37.8  , 37.79 , 37.797, 37.803, 32.416, 37.79 , 37.783, 37.821,
       37.767, 37.803, 32.258, 32.172, 32.258, 37.805, 37.833, 37.65 ,
       37.643, 37.575, 37.574, 37.707, 37.757, 37.696, 37.78 , 37.74 ,
      

<hr>

<h4>07 Create CSV</h4>

In [35]:
10/0
submission = pd.read_csv(r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\sample_submission.csv")
submission['23SoilWC ค.ชื้นดิน'].iloc[1:] = y[1:]
submission

ZeroDivisionError: division by zero